This is using Scikit-learn's GridSearchCV on XGBoost.

This is only meant to be an early rough experiment. See the caveats in README. Using a larger learning rate or a small dataset for better speed performance.

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
import warnings; warnings.simplefilter('ignore')
import inspect
from sklearn.metrics import roc_auc_score, make_scorer
roc = make_scorer(roc_auc_score)
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin

In [ ]:
class MyEstimator(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators = 100, learning_rate = 0.1, max_depth=3, objective='gpu:binary:logistic', gamma=0, min_child_weight=1, reg_alpha=0, reg_lambda=1            , random_state=2018, tree_method = 'gpu_hist', max_bin = 32, subsample =1, colsample_bytree = 1
 #              , eval_metric ='auc'
                ):
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")
        for arg, val in values.items():
            setattr(self, arg, val)   
        self.fitted_ = False
    
    def fit(self, X, y=None, sample_weight=None, evalset = None):
        self.booster_params = {'max_depth': self.max_depth, 'learning_rate': self.learning_rate, 
                               'objective': self.objective, 'gamma':self.gamma, 
                               'min_child_weight':self.min_child_weight, 'reg_alpha':self.reg_alpha,
                                'reg_lambda' : self.reg_lambda, 'seed' : self.random_state, 
                              'tree_method' : self.tree_method, 'max_bin': self.max_bin,
                              'subsample': self.subsample, 'colsample_bytree': self.colsample_bytree
                            #  ,'eval_metric':self.eval_metric
                              }
        xgtrain = xgb.DMatrix(data = X, label=y, weight = sample_weight)
        self.bst = xgb.train(params = self.booster_params, dtrain = xgtrain, 
                             num_boost_round = self.n_estimators
                  #  ,early_stopping_rounds = 50, evals = [(evalset, 'try')], maximize = True,verbose_eval = 50
                            )
        gc.enable()
        del xgtrain
        gc.collect()
        self.fitted_ = True
        return self

    def predict(self, X, y=None):   
        if not self.fitted_:
            raise RuntimeError("You must train classifer before predicting data!")
        xgtest = xgb.DMatrix(data = X)
        pred = self.bst.predict(xgtest)
        gc.enable()
        del xgtest
        gc.collect()
        return pred 

In [ ]:
train = pd.read_csv('train.csv', index_col = 0)
cols = list(train.columns)
cols.remove('TARGET')
tar = 'TARGET'

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)

gsparam = {'n_estimators' : [150], 'learning_rate' : [0.1], 'random_state' :[2018], 'max_depth' :[8], 'gamma' :[0.1], 'min_child_weight' : [45],  'reg_alpha':[0], 'reg_lambda':[3], 'max_bin':[32], 'subsample':[1], 'colsample_bytree':[1]}
           
wrapper = MyEstimator()
gs = GridSearchCV(estimator = wrapper, param_grid = gsparam, scoring = roc, cv = skf, return_train_score = False, verbose = 2, refit= False)
gs.fit(train[cols], train[tar])
pd.DataFrame(gs.cv_results_).to_csv('gridcv.csv')